# Text Prediction AI Model Training
This notebook will guide you through the process of training a text prediction AI model using GPT-2.


In [1]:
!pip install pandas transformers torch
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch


In [1]:
import torch
print(torch.cuda.is_available())  # Should return True if CUDA is available
print(torch.cuda.device_count())  # Should return the number of available GPUs
print(torch.cuda.get_device_name(0))  # Should return the name of the first GPU


False
0


AssertionError: Torch not compiled with CUDA enabled

In [2]:
import pandas as pd

data = pd.read_csv("dataset/ted_data.csv")

In [3]:
import re

# Function to clean the transcript text
def preprocess_text(text):
    text = re.sub(r'\(Laughter\)', '', text)  # Remove "(Laughter)"
    text = re.sub(r'\(Music:.*?\)', '', text)  # Remove "(Music: ...)"
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = text.strip()  # Remove leading and trailing spaces
    return text

# Apply the cleaning function to the 'transcript' column
data['cleaned_transcript'] = data['transcript'].apply(preprocess_text)


In [4]:
from transformers import AutoTokenizer

# Load the tokenizer (replace 'model-name' with the actual model name you are using)
tokenizer = AutoTokenizer.from_pretrained('gpt2')  # Example model name

# Set a padding token, you can use the end-of-sequence token or any other appropriate token
tokenizer.pad_token = tokenizer.eos_token

# Verify that the pad_token is set
print(f"Padding token: {tokenizer.pad_token}, ID: {tokenizer.pad_token_id}")


Padding token: <|endoftext|>, ID: 50256


In [5]:
# Function to tokenize the text
def tokenize_text(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=512)

# Apply the tokenization function to the 'cleaned_transcript' column
data['tokenized'] = data['cleaned_transcript'].apply(tokenize_text)


In [6]:
# Inspect the tokenized data
print(data[['cleaned_transcript', 'tokenized']].head())


                                  cleaned_transcript  \
0  Good morning. How are you?It's been great, has...   
1  Thank you so much, Chris. And it's truly a gre...   
2  Hello voice mail, my old friend.I've called fo...   
3  If you're here today — and I'm very happy that...   
4  About 10 years ago, I took on the task to teac...   

                     tokenized  
0  [input_ids, attention_mask]  
1  [input_ids, attention_mask]  
2  [input_ids, attention_mask]  
3  [input_ids, attention_mask]  
4  [input_ids, attention_mask]  


In [7]:
# Save the cleaned and tokenized data to a new CSV file (optional)
data.to_csv('dataset/cleaned_tokenized_ted_data.csv', index=False)


In [8]:
from datasets import Dataset

# Load the cleaned and tokenized data
data = pd.read_csv('dataset/cleaned_tokenized_ted_data.csv')

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(data)

In [9]:
!pip install transformers[torch] accelerate -U


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [10]:
!pip install -U 'accelerate==0.21.0'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached accelerate-0.21.0-py3-none-any.whl.metadata (17 kB)
Using cached accelerate-0.21.0-py3-none-any.whl (244 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1


In [11]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments

# Load the pre-trained model (replace 'model-name' with the actual model name you are using)
model_name = 'gpt2'  # Example model name
model = AutoModelForCausalLM.from_pretrained(model_name)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',           # output directory
    overwrite_output_dir=True,        # overwrite the content of the output directory
    num_train_epochs=3,               # number of training epochs
    per_device_train_batch_size=2,    # batch size for training
    save_steps=10_000,                # save checkpoint every 10,000 steps
    save_total_limit=2,               # only keep the last 2 checkpoints
    logging_dir='./logs',             # directory for storing logs
    logging_steps=200,                # log every 200 steps
)


In [12]:
from transformers import DataCollatorForLanguageModeling

# Define the data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # We are not using masked language modeling for GPT-2
)


In [13]:
# Function to map inputs to the required format
def tokenize_function(examples):
    return tokenizer(examples['cleaned_transcript'], truncation=True, padding='max_length', max_length=512)

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['transcript', 'url', 'cleaned_transcript'])


Map:   0%|          | 0/2467 [00:00<?, ? examples/s]

In [14]:
import accelerate

accelerate.__version__

'0.32.1'

In [15]:
!pip install pandas transformers[torch] accelerate -U datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached accelerate-0.32.1-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.32.1-py3-none-any.whl (314 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.21.0
    Uninstalling accelerate-0.21.0:
      Successfully uninstalled accelerate-0.21.0


In [16]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments

# Load the pre-trained model (replace 'model-name' with the actual model name you are using)
model_name = 'gpt2'  # Example model name
model = AutoModelForCausalLM.from_pretrained(model_name)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',           # output directory
    overwrite_output_dir=True,        # overwrite the content of the output directory
    num_train_epochs=3,               # number of training epochs
    per_device_train_batch_size=2,    # batch size for training
    save_steps=10_000,                # save checkpoint every 10,000 steps
    save_total_limit=2,               # only keep the last 2 checkpoints
    logging_dir='./logs',             # directory for storing logs
    logging_steps=200,                # log every 200 steps
)


In [17]:
from transformers import DataCollatorForLanguageModeling

# Define the data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # We are not using masked language modeling for GPT-2
)


In [18]:
# Function to map inputs to the required format
def tokenize_function(examples):
    return tokenizer(examples['cleaned_transcript'], truncation=True, padding='max_length', max_length=512)

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['transcript', 'url', 'cleaned_transcript'])


Map:   0%|          | 0/2467 [00:00<?, ? examples/s]

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Start training
trainer.train()
